In [15]:
import requests, io, os, time
import database as db
from datetime import datetime, timedelta
from typing import Union
from fastapi import Depends, FastAPI, HTTPException, status, File
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from jose import JWTError, jwt
from passlib.context import CryptContext
from pydantic import BaseModel
from PIL import Image

In [40]:
img = Image.open('img.jpg')
path = datetime.now().strftime("%Y%m")
os.makedirs(os.path.join('history', path), exist_ok=True)
img.save(os.path.join('history', path, str(time.time_ns()) + '.jpg'))
# s, type(s)

In [29]:
from mysql.connector import Error
from mysql.connector import pooling
import traceback

connection_pool = pooling.MySQLConnectionPool(pool_name="pynative_pool",
                                                  pool_size=5,
                                                  pool_reset_session=True,
                                                  host='localhost',
                                                  database='car-door-plate',
                                                  user='root',
                                                  password='180981')
print("Connection Pool Name - ", connection_pool.pool_name)
print("Connection Pool Size - ", connection_pool.pool_size)

Connection Pool Name -  pynative_pool
Connection Pool Size -  5


In [20]:
def create_user(username, password, refresh_token):
    conn = connection_pool.get_connection()
    cursor = conn.cursor()
    try:
        result = cursor.execute(f"""INSERT INTO `user` (`username`, `password`, `refresh_token`) 
                                VALUES ('{username}', '{password}', '{refresh_token}')""")
        conn.commit()
    except Error as e:
        traceback.print_exc()
        result = None
    finally:
        cursor.close()
        conn.close()
    
    return result

def get_user(username):
    conn = connection_pool.get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(f"SELECT * FROM `user` WHERE `username`='{username}'")
        result = cursor.fetchone()
        conn.commit()
    except Error:
        traceback.print_exc()
        return None
    finally:
        cursor.close()
        conn.close()
    return result

# create_user('hanv', '180981', 'abc')
print(get_user('hanv'))

(1, 'hanv', '180981', 'abc')


In [28]:
def create_user(username, password):
    response = requests.get(url='http://127.0.0.1:8000/create_user', 
                            params={'username': username, 'password':password})
    print(response.status_code, response.json())
create_user('hanv', '180981')

200 {'username': 'hanv', 'password': '180981'}


In [32]:
def add_history(username, plate, path):
    conn = connection_pool.get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(f"""INSERT INTO `history_{datetime.now().strftime("%Y%m")}` (`username`, `plate`, `path`)
                       VALUES ('{username}', '{plate}', '{path}')""")
        conn.commit()
    except Exception:
        print('Exception add_history')
    finally:
        cursor.close()
        conn.close()

add_history('hanv', 'abc123', 'history\\202310\\hanv.jpg')